In [34]:
from tqdm import tqdm
import pymssql
import pandas as pd
import numpy as np
import warnings
import glob 
import os

warnings.filterwarnings('ignore')

In [54]:
df = pd.read_csv(r'C:\Users\Trenkin.Sergey\Downloads\Telegram Desktop\result.csv')

In [95]:
df2 = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Петров Илья\sql\группировка.xlsx', header=1)

In [66]:
df2 = df2[['Unnamed: 0', 'min.5', 'max.5', 'median.5']]

In [67]:
df2.columns = ['Код товара', 'Наценка мин', 'Наценка макс', 'Наценка медиана']

In [3]:
os.chdir(r'C:\\Users\Trenkin.Sergey\Desktop\sales')
extension = 'xlsx'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames

['2024-03-10_2024-03-19.xlsx',
 '2024-03-20_2024-03-31.xlsx',
 '2024-04-01_2024-04-09.xlsx',
 '2024-04-10_2024-04-19.xlsx',
 '2024-04-20_2024-04-30.xlsx',
 '2024-05-01_2024-05-09.xlsx',
 '2024-05-10_2024-05-19.xlsx',
 '2024-06-01_2024-06-09.xlsx',
 '2024-06-10_2024-06-10.xlsx',
 '2024-06-10_2024-06-12.xlsx']

In [4]:
zapret = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\запрет Артикулы к размещению.xlsx')

In [5]:
images = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Артикул-фото 2024.06.11_.xlsx')

In [73]:
sale = pd.DataFrame()
for name in tqdm(all_filenames):
    sales = pd.read_excel(name)
    sale = pd.concat([sales, sale], ignore_index= True)

100%|██████████| 10/10 [09:44<00:00, 58.47s/it]


In [74]:
sale['Дата'] = pd.to_datetime(sale['Дата'], dayfirst=True, format='mixed')

In [75]:
sale = sale[(sale['Дата'] >= pd.to_datetime('2024-03-14', dayfirst=True, format='mixed')) & 
            (sale['Дата'] <= pd.to_datetime('2024-06-11', dayfirst=True, format='mixed'))]

In [77]:
sale = sale.groupby(by = 'Тов', as_index=False)[['Количество', 'Общая сумма']].sum()

In [80]:
sale = sale.query('Тов in @tovs')

In [82]:
sale.columns = ['Код товара', 'Количество продаж 14.03-11.06', 'Общая сумма продаж 14.03-11.06']

In [17]:
tovs = df['Код товара'].tolist()

In [23]:
zapret = zapret['Артикул сайта'].tolist()

In [60]:
df = df.dropna(subset=['ID Сайта'])
df['ID Сайта'] = df['ID Сайта'].astype(int)

In [61]:
df['Запрет к размещению'] = np.nan
for i in df.index:
    if df['ID Сайта'][i] in zapret:
        df['Запрет к размещению'][i] = True
    else:
        df['Запрет к размещению'][i] = False

In [62]:
df2.dropna(inplace = True)

In [69]:
df = df.drop_duplicates(subset=['Код товара'])

In [70]:
df = df.merge(df2, how='left', on='Код товара')

In [83]:
df = df.merge(sale, how='left', on='Код товара')

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138542 entries, 0 to 138541
Data columns (total 42 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Код товара                      138542 non-null  object 
 1   Артикул поставщика              138542 non-null  object 
 2   Внутренний артикул              138478 non-null  object 
 3   ID Сайта                        138542 non-null  int32  
 4   Товарное направление            138542 non-null  object 
 5   Товарная группа                 138479 non-null  object 
 6   Товарный кластер                138333 non-null  object 
 7   Проба                           138532 non-null  float64
 8   Размер изделия                  138542 non-null  float64
 9   Тип изделия 1                   138532 non-null  object 
 10  Тип изделия 2                   138479 non-null  object 
 11  Тип изделия 3                   138540 non-null  object 
 12  Дизайн          

In [ ]:
df2 = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Петров Илья\sql\группировка.xlsx', header=1)

In [97]:
df2 = df2[['Unnamed: 0', 'min.4', 'max.4', 'median.4']]

In [99]:
df2.columns = ['Код товара', 'Себестоимость мин', 'Себестоимость макс', 'Себестоимость медиана']

In [100]:
df2 = df2.drop_duplicates(subset=['Код товара'])

In [101]:
df = df.merge(df2, how='left', on='Код товара')

In [102]:
df['Себестоимость размах'] = (df['Себестоимость макс'] - df['Себестоимость мин']) / ((df['Себестоимость макс'] + df['Себестоимость мин']) / 2)

In [104]:
df.to_excel('Исследование цен.xlsx', index=False)